# 1. 데이터 필터

1. kofic 일별 박스오피스 데이터 (최종_박스오피스_데이터.csv)
2. cgv 줄거리 파일 (cgv_장르_줄거리_통합본.csv) - 기준
분석에 사용할 박스오피스 데이터만 걸러낸다

In [728]:
import pandas as pd
import numpy as np

In [729]:
kofic_data=pd.read_csv("../영화관람객크롤링/최종_박스오피스_데이터.csv",encoding='utf-8-sig')
cgv_data=pd.read_csv("../영화리뷰크롤링/cgv리뷰_전처리/전처리파일저장/cgv_장르_줄거리_통합본.csv",encoding='utf-8-sig')

In [730]:
kofic_data.head(1)

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분
0,#Take No. 고백(인디스데이),NaN,13000,2,1,1,한국,NaN,NaN,기타,NaN,NaN,2023-02-10,최초 개봉


In [731]:
cgv_data.head(1)

,영화제목,장르,줄거리
0,좋은날,애니메이션,"20대의 풋풋한 사랑 [봄봄], 40대의 처참했던 슬픔 [운수 좋은 날] 그리고 6..."


데이터 중복 확인

In [732]:
kofic_data[["영화명","일자"]].duplicated().sum()

np.int64(1894)

중복 데이터 1894개 제거

In [733]:
kofic_data.drop_duplicates(subset=["영화명","일자"],inplace=True)

# 2. 사용할 영화

In [734]:
barometer=cgv_data["영화제목"]
temp=kofic_data[kofic_data["영화명"].isin(barometer)]

In [735]:
temp["영화명"].nunique()

6472

# 3. 최초개봉만

In [736]:
temp=temp[temp["개봉 구분"]=="최초 개봉"]
temp.head()

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분
12,#살아있다,2020-06-24,994000,142,1,1,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-02,최초 개봉
13,#살아있다,2020-06-24,909000,449,2,2,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-03,최초 개봉
14,#살아있다,2020-06-24,7314000,1386,7,11,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-15,최초 개봉
15,#살아있다,2020-06-24,3950000,790,6,6,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-19,최초 개봉
16,#살아있다,2020-06-24,2503000,1239,5,5,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-23,최초 개봉


# 4. n주차 생성

개봉일이 없는 영화 - 그 영화는 모든 일자에 개봉일이 없나?  
개봉일이 하나도 없는 영화는 제거

In [737]:
temp = temp[temp.groupby('영화명')['개봉일'].transform('count') > 0]

In [738]:
temp.isna().sum()

영화명          0
개봉일         86
매출액          0
관객수          0
스크린수         0
         ...  
장르         143
감독        7709
배우       16323
일자           0
개봉 구분        0
Length: 14, dtype: int64

개봉일이 있는것과 없는 영화도 있음  
결측치 보간

In [739]:
temp['개봉일'] = temp.groupby('영화명')['개봉일'].transform('first')

In [740]:
temp.isna().sum()

영화명          0
개봉일          0
매출액          0
관객수          0
스크린수         0
         ...  
장르         143
감독        7709
배우       16323
일자           0
개봉 구분        0
Length: 14, dtype: int64

날짜 형식 변경  
  
1주차: 영화 개봉일이 포함된 주의 일요일까지  
2주차 이후: 월요일부터 일요일까지를 한 주로 계산

In [741]:
temp['개봉일_dt'] = pd.to_datetime(temp['개봉일'], errors='coerce')
temp['일자_dt'] = pd.to_datetime(temp['일자'], errors='coerce')

In [742]:
temp.head()

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분,개봉일_dt,일자_dt
12,#살아있다,2020-06-24,994000,142,1,1,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-02,최초 개봉,2020-06-24,2020-06-02
13,#살아있다,2020-06-24,909000,449,2,2,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-03,최초 개봉,2020-06-24,2020-06-03
14,#살아있다,2020-06-24,7314000,1386,7,11,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-15,최초 개봉,2020-06-24,2020-06-15
15,#살아있다,2020-06-24,3950000,790,6,6,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-19,최초 개봉,2020-06-24,2020-06-19
16,#살아있다,2020-06-24,2503000,1239,5,5,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-23,최초 개봉,2020-06-24,2020-06-23


일자가 개봉일보다 빠른 것은 제거 (개봉전 시사회일 가능성 높음)

In [743]:
temp=temp[temp['개봉일_dt'] <= temp['일자_dt']]

각 영화별로 고유한 첫 개봉일을 '개봉일_대표' 컬럼으로 지정 (날짜 형식 변경한 것을 사용)

In [744]:
temp['개봉일_대표'] = temp.groupby('영화명')['개봉일_dt'].transform('first')

개봉일이 속한 주의 일요일을 '첫주_마감일'로 계산  
월요일 : 0, 일요일 : 6

In [745]:
days_to_sunday = 6 - temp['개봉일_대표'].dt.weekday
temp['첫주_마감일'] = temp['개봉일_대표'] + pd.to_timedelta(days_to_sunday, unit='D')

주차 계산

In [746]:
# 일자_dt가 첫주_마감일보다 작거나 같으면 1일차
is_first_week = temp['일자_dt'] <= temp['첫주_마감일']
# 일자_dt와 첫주_마감일의 차이를 일수로 계산
days_diff = (temp['일자_dt'] - temp['첫주_마감일']).dt.days
# 2주차 부터의 주차 계산
weeks_after_first: int = (days_diff - 1) // 7 + 2 # 값 자체가 주차 숫자
# 첫주차면 1, 아니면 계산된 주차 숫자를 입력
temp['주차'] = np.where(is_first_week, 1, weeks_after_first)

이제 필요없는 컬럼 삭제

In [747]:
temp.drop(columns=['개봉일_dt', '일자_dt', '개봉일_대표', '첫주_마감일'], inplace=True)

확인

In [748]:
temp.tail(5)

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분,주차
682460,힛쳐,2007-08-01,54000,8,2,3,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-08,최초 개봉,2
682461,힛쳐,2007-08-01,54000,8,1,2,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-10,최초 개봉,2
682462,힛쳐,2007-08-01,26000,4,1,2,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-11,최초 개봉,2
682463,힛쳐,2007-08-01,60000,9,1,2,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-12,최초 개봉,2
682464,힛쳐,2007-08-01,7000,1,1,1,미국,(주)씨지브이신도림,청소년관람불가,"액션,범죄,스릴러",데이브 메이어스,"숀 빈,소피아 부시,자크리 나이튼,닐 맥도노프",2007-08-13,최초 개봉,3


In [749]:
temp[temp["영화명"]=="#살아있다"].groupby("주차").size()

주차
1     5
2     7
3     7
4     7
5     7
     ..
7     7
8     7
9     5
10    3
19    1
Length: 11, dtype: int64

In [750]:
pd.set_option('display.max_rows', 100)
check_temp = temp.groupby(['영화명', '주차']).size().reset_index(name='counts')
check_temp.head(100)

,영화명,주차,counts
0,#살아있다,1,5
1,#살아있다,2,7
2,#살아있다,3,7
3,#살아있다,4,7
4,#살아있다,5,7
5,#살아있다,6,7
6,#살아있다,7,7
7,#살아있다,8,7
8,#살아있다,9,5
9,#살아있다,10,3


In [751]:
pd.set_option('display.max_rows', 10)

각 주차별 일자가 7일이 넘는지 확인 (데이터 중복 검사)

In [752]:
check_temp[check_temp['counts'] > 7]

,영화명,주차,counts


문제점  
1. 주차가 확 띄는 경우가 있다 -> 재개봉이 잘 걸러지지 않았다, 따라서 이를 제거  
2. 중간중간 데이터가 비는 경우가 있다 -> 영화별로 얼만큼 비는지 확인

재개봉 주차를 제거  
마지막 개봉주차와의 차이가 1보다 크면 재개봉이라고 판단

In [753]:
check_temp['주차_차이'] = check_temp.groupby('영화명')['주차'].diff()

1~7, 13, 14주차와 같은 경우 14주차는 분명 재개봉이지만 주차 차이는 1이다  
이것은 누적합으로 해결한다

이 방법대로 하면 1~8, 10~주차의 경우 재개봉이 아님에도 판별 못할 수 있다  
그러나 이정도 차이가 나면 결측치 보간을 할때 그 정확도가 매우 떨어진다  
따라서 해당 영화를 아예 제거하는 것보다, 일부분만이라도 사용하는게 나을 수 있지 않을까?  
(어차피 인기있는 영화면 박스오피스 데이터가 누락될 일이 없을 것)

In [754]:
check_temp["재개봉_시작_주차"] = check_temp.groupby('영화명')['주차'].diff()>1 #재개봉 시작 주차 표시
check_temp["재개봉_누적"] = check_temp.groupby("영화명")["재개봉_시작_주차"].cumsum()


In [755]:
pd.set_option('display.max_rows', 1000)
check_temp.head(1000)

,영화명,주차,counts,주차_차이,재개봉_시작_주차,재개봉_누적
0,#살아있다,1,5,NaN,False,0
1,#살아있다,2,7,1.0,False,0
2,#살아있다,3,7,1.0,False,0
3,#살아있다,4,7,1.0,False,0
4,#살아있다,5,7,1.0,False,0
5,#살아있다,6,7,1.0,False,0
6,#살아있다,7,7,1.0,False,0
7,#살아있다,8,7,1.0,False,0
8,#살아있다,9,5,1.0,False,0
9,#살아있다,10,3,1.0,False,0


In [756]:
pd.set_option('display.max_rows', 10)

In [757]:
check_temp=check_temp[check_temp['재개봉_누적'] == 0]

본 분석은 2주차 까지의 데이터를 사용한다.  
반대로 말하면, 2주차 이후의 데이터가 없으면 해당 영화는 사용할 수 없다

In [758]:
check_temp.shape

(37771, 6)

In [759]:
# '주차' 열에 2가 포함된 영화명을 추출합니다.
movies_with_week_2 = check_temp[check_temp['주차'] == 2]['영화명'].unique()

# 전체 영화명 목록을 가져옵니다.
all_movies = check_temp['영화명'].unique()

# 전체 영화 목록에서 2주차 데이터가 있는 영화 목록을 제외합니다.
movies_without_week_2 = [movie for movie in all_movies if movie not in movies_with_week_2]

# 최종 개수를 출력합니다.
print(len(movies_without_week_2))

492


In [760]:
pd.set_option('display.max_rows', 100)
wk_2 = check_temp[check_temp["영화명"].isin(movies_without_week_2)].groupby(["영화명", "주차"]).size().reset_index(name='counts')
wk_2.head(100)

,영화명,주차,counts
0,2024 태민 월드 투어 [이페머럴 게이즈],1,1
1,28일 후,826,1
2,28일 후,827,1
3,28일 후,828,1
4,28일 후,829,1
5,3670,1,1
6,4인용 식탁,346,1
7,7인의 황후,1,1
8,8 마일,40,1
9,8 마일,41,1


확인 결과 : 최초개봉일과 일자 차이가 많다 -> 박스오피스 계측 전(2003년 11월) 이전의 영화의 관람객 데이터는 없다. 이것이 후에 재개봉하면 자연스레 주차수가 비정상적으로 커진다

In [761]:
check_temp = check_temp[~check_temp["영화명"].isin(movies_without_week_2)]
check_temp.head(20)

,영화명,주차,counts,주차_차이,재개봉_시작_주차,재개봉_누적
0,#살아있다,1,5,NaN,False,0
1,#살아있다,2,7,1.0,False,0
2,#살아있다,3,7,1.0,False,0
3,#살아있다,4,7,1.0,False,0
4,#살아있다,5,7,1.0,False,0
5,#살아있다,6,7,1.0,False,0
6,#살아있다,7,7,1.0,False,0
7,#살아있다,8,7,1.0,False,0
8,#살아있다,9,5,1.0,False,0
9,#살아있다,10,3,1.0,False,0


In [762]:
pd.set_option('display.max_rows', 10)

# 5. 주차관련 오류 최종 수정

이제 해당 영화명과 주차를 가지고 temp에서 데이터를 최종 필터링한다

In [763]:
result = pd.merge(temp, check_temp[["영화명","주차"]], how="inner")
result.head(10)

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자,개봉 구분,주차
0,#살아있다,2020-06-24,1311296000,204066,1662,7223,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-24,최초 개봉,1
1,#살아있다,2020-06-24,1240607900,148991,1637,7311,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-25,최초 개봉,1
2,#살아있다,2020-06-24,1380200100,152550,1713,7839,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-26,최초 개봉,1
3,#살아있다,2020-06-24,2644122800,294255,1882,9221,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-27,최초 개봉,1
4,#살아있다,2020-06-24,2263447540,256133,1869,9044,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-28,최초 개봉,1
5,#살아있다,2020-06-24,556892400,67845,1644,7111,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-29,최초 개봉,2
6,#살아있다,2020-06-24,543848720,67098,1641,7147,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-06-30,최초 개봉,2
7,#살아있다,2020-06-24,428624500,51659,1441,6137,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-07-01,최초 개봉,2
8,#살아있다,2020-06-24,368804680,44470,1401,5783,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-07-02,최초 개봉,2
9,#살아있다,2020-06-24,467173540,51860,1444,6066,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",2020-07-03,최초 개봉,2


In [764]:
pd.set_option('display.max_rows', 100)
result.groupby(["영화명","주차"]).size().reset_index(name='counts').head(100)

,영화명,주차,counts
0,#살아있다,1,5
1,#살아있다,2,7
2,#살아있다,3,7
3,#살아있다,4,7
4,#살아있다,5,7
5,#살아있다,6,7
6,#살아있다,7,7
7,#살아있다,8,7
8,#살아있다,9,5
9,#살아있다,10,3


In [765]:
pd.set_option('display.max_rows', 10)

In [766]:
result["영화명"].nunique()

4773

최종 4773개의 영화가 추출되었다

# 6. 공휴일 등 날짜 결합

공휴일과 날짜가 있는 데이터를 일자를 기준으로 결합한다

In [767]:
date_data = pd.read_csv("../calendar_dataset.csv", encoding='utf-8-sig')
date_data.head()

,Date,Holiday,Moonhwa
0,2003-01-01,1,0
1,2003-01-02,0,0
2,2003-01-03,0,0
3,2003-01-04,1,0
4,2003-01-05,1,0


In [768]:
date_data.isna().sum() 

Date       0
Holiday    0
Moonhwa    0
dtype: int64

In [769]:
date_data["Date"] = pd.to_datetime(date_data["Date"], errors='coerce')
date_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8340 entries, 0 to 8339
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     8340 non-null   datetime64[ns]
 1   Holiday  8340 non-null   int64         
 2   Moonhwa  8340 non-null   int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 195.6 KB


In [770]:
result["일자"] = pd.to_datetime(result["일자"], errors='coerce')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197697 entries, 0 to 197696
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   영화명     197697 non-null  object        
 1   개봉일     197697 non-null  object        
 2   매출액     197697 non-null  int64         
 3   관객수     197697 non-null  int64         
 4   스크린수    197697 non-null  int64         
 5   상영횟수    197697 non-null  int64         
 6   대표국적    197697 non-null  object        
 7   배급사     197322 non-null  object        
 8   등급      197631 non-null  object        
 9   장르      197578 non-null  object        
 10  감독      191135 non-null  object        
 11  배우      184300 non-null  object        
 12  일자      197697 non-null  datetime64[ns]
 13  개봉 구분   197697 non-null  object        
 14  주차      197697 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(9)
memory usage: 22.6+ MB


일자 기준으로 병합

In [771]:
result.shape

(197697, 15)

In [772]:
temp_result = pd.merge(result, date_data, left_on = "일자", right_on = "Date", how = "left")
temp_result.shape


(197697, 18)

In [773]:
temp_result.drop(columns=["일자"], inplace=True)
temp_result.head(2)

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,개봉 구분,주차,Date,Holiday,Moonhwa
0,#살아있다,2020-06-24,1311296000,204066,1662,7223,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",최초 개봉,1,2020-06-24,0,1
1,#살아있다,2020-06-24,1240607900,148991,1637,7311,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,드라마,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",최초 개봉,1,2020-06-25,0,0


`펜데믹 구분` 은 아래와 같은 규칙으로 생성한다

1. 2020년 1월 까지는 `0`
2. 2020년 2월부터 2023년 4월까지는 `1`
3. 2023년 5월부터는 `2`

In [774]:
def get_pandemic_flag(date):
    if date < pd.Timestamp('2020-02-01'):
        return 0
    elif date < pd.Timestamp('2023-05-01'):
        return 1
    else:
        return 2

temp_result['Pandemic'] = temp_result['Date'].apply(get_pandemic_flag)

# 7. 최종 사용 컬럼 및 영문명 변경

In [777]:
final_result = \
temp_result.rename(columns={
    "영화명": "Movie_Title",
    "개봉일" : "Release_Date",
    "관객수": "Audience_Count",
    "상영횟수":"Show_Count",
    "대표국적":"Main_Country",
    "배급사":"Distributor",
    "등급":"Grade",
    "장르":"Genre",
    "주차":"Week"
    })

In [779]:
final_result.drop(columns=["매출액", "스크린수", "감독", "배우", "개봉 구분"], inplace=True)

In [780]:
final_result.to_csv("boxoffice_data.csv", index=False, encoding='utf-8-sig')